In [57]:
import pandas as pd
import numpy as np

data = r"3993249.csv"

df = pd.read_csv(data)
df.set_index('DATE', inplace=True)

df = df[df['STATION'] != 'USR0000NBES'] # get rid of Bessey Station, it doesn't have much data
stations = df['STATION'].unique()

df = df[['STATION', 'PRCP', 'TMAX', 'TMIN']]

df = df.pivot(columns='STATION', values=['PRCP', 'TMAX', 'TMIN'])
df = df.loc['1962-07-01':'2024-12-31']
df = df.ffill()

df['PRCP'] *= 100.0

df.columns = ['_'.join(col) for col in df.columns]

df.insert(0, 'DAY', pd.to_datetime(df.index).day_of_year)
df['DAY'] = df['DAY'].apply(lambda x: x if x < 366 else 1)
df['DAY365'] = df['DAY'] / 365.25
df['SIN'] = np.sin(2 * np.pi * df['DAY365'])
df['COS'] = np.cos(2 * np.pi * df['DAY365'])

df.to_csv('multidata.csv')

# for station in stations:
#     station_prcp = f'PRCP_{station}'
#     station_tmax = f'TMAX_{station}'
#     station_tmin = f'TMIN_{station}'
    
#     station_tspread = f'TSPREAD_{station}'
#     station_pspread = f'PSPREAD_{station}'
#     df[station_tspread] = df[station_tmax] - df[station_tmin]
#     df[station_pspread] = df[station_tspread] * df[station_prcp] / 100.0

#     station_prcp_sin

In [58]:
limited = df.loc[:'2000-01-01']

normals = limited.groupby('DAY').agg(['mean', 'std'])
normals.columns = ['_'.join(col) for col in normals.columns]
normals.to_csv('multinormals.csv')